# Initial problem resolution

In [2]:
using JuMP
using Gurobi
using CSV
using DataFrames

In [3]:
# Load time series data
load_data = CSV.read("county_demand_local_hourly_2014.csv", DataFrame)
solar_cf = CSV.read("cf_local_county_2014.csv", DataFrame)

# Load network data
bus_data = CSV.read("IEEE_33_bus_system.csv", DataFrame)
max_flows = CSV.read("max_flows.csv", DataFrame)


rename!(max_flows, "Line number" => "Branch No.")
network_data_match = all(eachrow(bus_data[!, ["From bus", "To bus", "Branch No."]]) .== eachrow(max_flows[!, ["From bus", "To bus", "Branch No."]]))

if network_data_match
    # Merge the data
    network_data = innerjoin(bus_data, max_flows, on=["From bus", "To bus", "Branch No."])
    println("Data merged successfully")
else
    println("Data columns do not match")
end

buses = 1:33
lines = 1:32
T = 8760;

Data merged successfully


# First Paper


In [40]:
# Lossless power flow model
function create_hc_model_naive(network_data)
    model = Model(Gurobi.Optimizer)

    # Variables
    @variable(model, PG[buses] >= 0)  # Active power generation of PV
    @variable(model, QG[buses])  # Reactive power generation of PV
    @variable(model, -0.1 <= ΔV[buses] <= 0.1)  # Voltage magnitude deviation
    @variable(model, Δθ[buses])  # Voltage angle deviation
    @variable(model, PL[lines])  # Active power flow
    @variable(model, QL[lines])  # Reactive power flow
    @variable(model, PM)  # Active power exchange with upstream grid
    @variable(model, QM)  # Reactive power exchange with upstream grid

    # Objective function
    @objective(model, Max, sum(PG))

    # Constraints
    # Power flow equations
    for l in lines
        i, j = network_data[l, Symbol("From bus")], network_data[l, Symbol("To bus")]
        r, x = network_data[l, Symbol("R (Ω)")], network_data[l, Symbol("X (Ω)")]
        g, b = r / (r^2 + x^2), -x / (r^2 + x^2)
        @constraint(model, PL[l] == g * (1 + ΔV[i]) * (ΔV[i] - ΔV[j]) - b * (Δθ[i] - Δθ[j]))
        @constraint(model, QL[l] == -b * (1 + ΔV[i]) * (ΔV[i] - ΔV[j]) - g * (Δθ[i] - Δθ[j]))
    end

    # Power balance constraints
    for i in buses
        @constraint(model, 
            (i == 1 ? PM : 0) + 
            sum(PL[l] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(PL[l] for l in lines if network_data[l, Symbol("From bus")] == i) + 
            PG[i] == 
            sum(network_data[l, Symbol("P (kW)")] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Convert kW to MW
        )
        @constraint(model, 
            (i == 1 ? QM : 0) + 
            sum(QL[l] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(QL[l] for l in lines if network_data[l, Symbol("From bus")] == i) +
            QG[i] == 
            sum(network_data[l, Symbol("Q (kW)")] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Convert kW to MW
        )
    end

    @constraint(model, PG[1] == 0)  
    @constraint(model, QG[1] == 0)

    # Voltage and line capacity constraints
    @constraint(model, -4.6 .<= PM .<= 4.6)  # Active power exchange limits with upstream grid
    @constraint(model, -4.6 .<= QM .<= 4.6)  # Reactive power exchange limits with upstream grid
    for l in lines
        active_max_flow = network_data[l, Symbol("Maximum line capacity (active power [KW])")] / 1000  # Convert to MW
        reactive_max_flow = network_data[l, Symbol("Maximum line capacity (reactive power [KVAR])")] / 1000  # Convert to MVAR
        @constraint(model, -active_max_flow <= PL[l] <= active_max_flow)
        @constraint(model, -reactive_max_flow <= QL[l] <= reactive_max_flow)
    end

    return model
end


function main_naive()
    model = create_hc_model_naive(network_data)
    optimize!(model)

    # Store all solution values
    PG_values = Array(value.(model[:PG]))
    QG_values = Array(value.(model[:QG]))
    ΔV_values = Array(value.(model[:ΔV]))
    Δθ_values = Array(value.(model[:Δθ]))
    PL_values = Array(value.(model[:PL]))
    QL_values = Array(value.(model[:QL]))
    PM_value = value(model[:PM])
    QM_value = value(model[:QM])

    # Create dataframes to display the results
    bus_results = DataFrame(Bus = buses, PG = PG_values, QG = QG_values, ΔV = ΔV_values, Δθ = Δθ_values)
    line_results = DataFrame(Line = lines, PL = PL_values, QL = QL_values)

    # Display the results
    println("Bus Results:")
    println(bus_results)
    println("\nLine Results:")
    println(line_results)
    println("\nUpstream Grid Exchange:")
    println("PM: ", PM_value)
    println("QM: ", QM_value)

    # Print results
    println("Optimal PV capacity: ", value.(model[:PG]))
    println("Total PV capacity: ", sum(value.(model[:PG])))
end
function optimize_naive_model(network_data)
    model = create_hc_model_naive(network_data)
    optimize!(model)
    return model
end

function display_naive_results(model)
    # Store all solution values
    PG_values = Array(value.(model[:PG]))
    QG_values = Array(value.(model[:QG]))
    ΔV_values = Array(value.(model[:ΔV]))
    Δθ_values = Array(value.(model[:Δθ]))
    PL_values = Array(value.(model[:PL]))
    QL_values = Array(value.(model[:QL]))
    PM_value = value(model[:PM])
    QM_value = value(model[:QM])

    # Create dataframes to display the results
    bus_results = DataFrame(Bus = buses, PG = PG_values, QG = QG_values, ΔV = ΔV_values, Δθ = Δθ_values)
    line_results = DataFrame(Line = lines, PL = PL_values, QL = QL_values)

    # Display the results
    println("\nUpstream Grid Exchange:")
    println("PM: ", PM_value)
    println("QM: ", QM_value)
    println("Total PV capacity: ", sum(PG_values[k] for k in buses))

    println("Bus Results:")
    println(bus_results)
    println("\nLine Results:")
    println(line_results)
    
end

display_naive_results (generic function with 1 method)

In [26]:
model = optimize_naive_model(network_data)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 134 rows, 264 columns and 330 nonzeros
Model fingerprint: 0x149c2366
Model has 64 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-01, 9e+00]
  QLMatrix range   [3e-01, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 5e+00]
  RHS range        [1e-02, 6e-01]
Presolve removed 106 rows and 106 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 74 rows and 134 columns
Presolve time: 0.00s
Presolved: 307 rows, 191 columns, 940 nonzeros
Presolved model has 61 bilinear constraint(s)
Variable types: 191 continuous, 0 integer (0 binary)
Found

A JuMP Model
├ solver: Gurobi
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 198
├ num_constraints: 297
│ ├ AffExpr in MOI.EqualTo{Float64}: 68
│ ├ AffExpr in MOI.Interval{Float64}: 66
│ ├ QuadExpr in MOI.EqualTo{Float64}: 64
│ ├ VariableRef in MOI.GreaterThan{Float64}: 66
│ └ VariableRef in MOI.LessThan{Float64}: 33
└ Names registered in the model
  └ :PG, :PL, :PM, :QG, :QL, :QM, :ΔV, :Δθ

In [41]:
display_naive_results(model)


Upstream Grid Exchange:
PM: -4.3321041165216
QM: 4.6
Total PV capacity: 8.017104116521597
Bus Results:
33×5 DataFrame
 Row │ Bus    PG         QG           ΔV            Δθ        
     │ Int64  Float64    Float64      Float64       Float64   
─────┼────────────────────────────────────────────────────────
   1 │     1  0.0         0.0         -0.1          -5.28384
   2 │     2  2.76921    -1.01353      0.1          -4.65307
   3 │     3  0.389469   -1.28845     -0.00672097   -2.69316
   4 │     4  0.276476   -0.28411      0.00395999   -2.03648
   5 │     5  0.230824   -0.476809     0.00682544   -1.51241
   6 │     6  0.21905    -0.40257      0.000326802  -0.757614
   7 │     7  0.211789    0.166282    -0.00119459   -0.53354
   8 │     8  0.176239    0.133571    -0.00179032    0.113847
   9 │     9  0.151251    0.195627    -0.0016853     0.555036
  10 │    10  0.128587   -0.171196    -0.019645      1.13458
  11 │    11  0.116584    0.110754     0.00470228    1.18323
  12 │    12  0.10

In [ ]:
function create_hc_model_robust(network_data, PD_min, PD_max, QD_min, QD_max)
    model = Model(Gurobi.Optimizer)

    # Variables
    @variable(model, PG[buses] >= 0)  # Active power generation of PV
    @variable(model, QG[buses])  # Reactive power generation of PV
    @variable(model, -0.1 <= ΔV[buses] <= 0.1)  # Voltage magnitude deviation
    @variable(model, Δθ[buses])  # Voltage angle deviation
    @variable(model, PL[lines])  # Active power flow
    @variable(model, QL[lines])  # Reactive power flow
    @variable(model, PM)  # Active power exchange with upstream grid
    @variable(model, QM)  # Reactive power exchange with upstream grid

    # Objective function
    @objective(model, Max, sum(PG))  # Maximize total PV generation

    # Constraints
    # Power flow equations
    for l in lines
        i, j = network_data[l, Symbol("From bus")], network_data[l, Symbol("To bus")]
        r, x = network_data[l, Symbol("R (Ω)")], network_data[l, Symbol("X (Ω)")]
        g, b = r / (r^2 + x^2), -x / (r^2 + x^2)
        @constraint(model, PL[l] == g * (1 + ΔV[i]) * (ΔV[i] - ΔV[j]) - b * (Δθ[i] - Δθ[j]))
        @constraint(model, QL[l] == -b * (1 + ΔV[i]) * (ΔV[i] - ΔV[j]) - g * (Δθ[i] - Δθ[j]))
    end

    # Power balance constraints with uncertainty
    for i in buses
        @constraint(model, 
            (i == 1 ? PM : 0) + 
            sum(PL[l] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(PL[l] for l in lines if network_data[l, Symbol("From bus")] == i) + 
            PG[i] >= sum(PD_min[l] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Lower bound on active power demand
        )
        @constraint(model, 
            (i == 1 ? PM : 0) + 
            sum(PL[l] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(PL[l] for l in lines if network_data[l, Symbol("From bus")] == i) + 
            PG[i] <= sum(PD_max[l] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Lower bound on active power demand
        )
        @constraint(model, 
            (i == 1 ? QM : 0) + 
            sum(QL[l] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(QL[l] for l in lines if network_data[l, Symbol("From bus")] == i) + 
            QG[i] >= sum(QD_min[l] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Lower bound on active power demand
        )
        @constraint(model, 
            (i == 1 ? QM : 0) + 
            sum(QL[l] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(QL[l] for l in lines if network_data[l, Symbol("From bus")] == i) + 
            QG[i] <= sum(QD_max[l] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Lower bound on active power demand
        )
    end

    @constraint(model, PG[1] == 0)  
    @constraint(model, QG[1] == 0)

    # Voltage and line capacity constraints
    @constraint(model, -4.6 .<= PM .<= 4.6)  # Active power exchange limits with upstream grid
    @constraint(model, -4.6 .<= QM .<= 4.6)  # Reactive power exchange limits with upstream grid
    for l in lines
        active_max_flow = network_data[l, Symbol("Maximum line capacity (active power [KW])")] / 1000  # Convert to MW
        reactive_max_flow = network_data[l, Symbol("Maximum line capacity (reactive power [KVAR])")] / 1000  # Convert to MVAR
        @constraint(model, -active_max_flow <= PL[l] <= active_max_flow)
        @constraint(model, -reactive_max_flow <= QL[l] <= reactive_max_flow)
    end

    return model
end

function optimize_robust()
    # Define uncertainty sets
    PD_min = [0.8 * p for p in network_data[:, Symbol("P (kW)")]]
    PD_max = [1.2 * p for p in network_data[:, Symbol("P (kW)")]]
    QD_min = [0.8 * q for q in network_data[:, Symbol("Q (kW)")]]
    QD_max = [1.2 * q for q in network_data[:, Symbol("Q (kW)")]]
    
    model = create_hc_model_robust(network_data, PD_min, PD_max, QD_min, QD_max)
    optimize!(model)
    
    return model
end

function display_robust_results(model)
    # Store all solution values
    PG_values = value.(model[:PG])
    QG_values = value.(model[:QG])
    ΔV_values = value.(model[:ΔV])
    Δθ_values = value.(model[:Δθ])
    PL_values = value.(model[:PL])
    QL_values = value.(model[:QL])
    PM_value = value(model[:PM])
    QM_value = value(model[:QM])

    # Convert DenseAxisArray to regular arrays
    PG_values_array = collect(PG_values)
    QG_values_array = collect(QG_values)
    ΔV_values_array = collect(ΔV_values)
    Δθ_values_array = collect(Δθ_values)
    PL_values_array = collect(PL_values)
    QL_values_array = collect(QL_values)

    # Create dataframes to display the results
    bus_results = DataFrame(Bus = buses, PG = PG_values_array, QG = QG_values_array, ΔV = ΔV_values_array, Δθ = Δθ_values_array)
    line_results = DataFrame(Line = lines, PL = PL_values_array, QL = QL_values_array)

    # Display the results
    println("\nUpstream Grid Exchange:")
    println("PM: ", PM_value)
    println("QM: ", QM_value)
    println("Total PV capacity: ", sum(PG_values_array))

    println("Bus Results:")
    println(bus_results)
    println("\nLine Results:")
    println(line_results)
end

main_robust (generic function with 1 method)

In [ ]:
model = optimize_robust()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 200 rows, 264 columns and 526 nonzeros
Model fingerprint: 0xf4ae060f
Model has 64 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [3e-01, 9e+00]
  QLMatrix range   [3e-01, 9e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 5e+00]
  RHS range        [8e-03, 7e-01]
Presolve removed 172 rows and 106 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 140 rows and 134 columns
Presolve time: 0.00s
Presolved: 307 rows, 191 columns, 940 nonzeros
Presolved model has 61 bilinear constraint(s)
Variable types: 191 continuous, 0 integer (0 binary)
Foun

In [ ]:
display_robust_results(model)

# Stochastic Paper

In [8]:
# Lossless power flow model
function create_hc_model(load_data, solar_cf, network_data)
    model = Model(Gurobi.Optimizer)

    # Variables
    @variable(model, CPV[buses] >= 0)  # PV capacity at each bus
    @variable(model, -1 <= ΔV[buses, 1:T] <= 1)  # Voltage magnitude deviation
    @variable(model, Δθ[buses, 1:T])  # Voltage angle deviation
    @variable(model, PL[lines, 1:T])  # Active power flow
    @variable(model, QL[lines, 1:T])  # Reactive power flow
    @variable(model, PM[1:T])  # Active power exchange with upstream grid
    @variable(model, QM[1:T])  # Reactive power exchange with upstream grid

    # Objective function
    @objective(model, Max, sum(CPV))

    # Constraints
    # Power flow equations
    for t in 1:T, l in lines
        i, j = network_data[l, Symbol("From bus")], network_data[l, Symbol("To bus")]
        r, x = network_data[l, Symbol("R (Ω)")], network_data[l, Symbol("X (Ω)")]
        g, b = r / (r^2 + x^2), -x / (r^2 + x^2)
        @constraint(model, PL[l,t] == g * (1 + ΔV[i,t]) * (ΔV[i,t] - ΔV[j,t]) - b * (Δθ[i,t] - Δθ[j,t]))
        @constraint(model, QL[l,t] == -b * (1 + ΔV[i,t]) * (ΔV[i,t] - ΔV[j,t]) - g * (Δθ[i,t] - Δθ[j,t]))
    end

    # Power balance constraints
    for t in 1:T, i in buses
        @constraint(model, 
            (i == 1 ? PM[t] : 0) + 
            sum(PL[l,t] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(PL[l,t] for l in lines if network_data[l, Symbol("From bus")] == i) + 
            solar_cf[t, Symbol("09003")] * CPV[i] == 
            load_data[t, Symbol("9003")] / 33  # Assuming equal load distribution among buses
        )
        @constraint(model, 
            (i == 1 ? QM[t] : 0) + 
            sum(QL[l,t] for l in lines if network_data[l, Symbol("To bus")] == i) - 
            sum(QL[l,t] for l in lines if network_data[l, Symbol("From bus")] == i) == 
            sum(network_data[l, Symbol("Q (kW)")] for l in lines if network_data[l, Symbol("To bus")] == i) / 1000  # Convert kVAR to MVAR
        )
    end

    @constraint(model, CPV[1] == 0)  

    # Voltage and line capacity constraints
    for t in 1:T
        @constraint(model, -0.1 .<= ΔV[:, t] .<= 0.1)  # Voltage deviation limits
        @constraint(model, -4.6 .<= PM[t] .<= 4.6)  # Active power exchange limits with upstream grid
        for l in lines
            max_flow = network_data[l, Symbol("Maximum line capacity (reactive power [KW])")] / 1000  # Convert to MW
            @constraint(model, -max_flow <= PL[l,t] <= max_flow)
            @constraint(model, -max_flow <= QL[l,t] <= max_flow)
        end
    end


    # # Net load deviation constraint
    # nam = 24  # Number of hours to consider for net load deviation
    # for t in 1:(T-nam)
    #     @constraint(model, 
    #         abs(sum(PM[t+h] for h in 1:nam) / nam - sum(PM[t+h-nam] for h in 1:nam) / nam) <= 0.1 * maximum(network_data[:, Symbol("Maximum line capacity (reactive power [KW])")]) / 1000
    #     )
    # end

    return model
end


function main()
    model = create_hc_model(load_data, solar_cf, network_data)
    optimize!(model)

    # Print results
    println("Optimal PV capacity: ", value.(model[:CPV]))
    println("Total PV capacity: ", sum(value.(model[:CPV])))
end

main (generic function with 1 method)

In [9]:
main()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-09-06


ArgumentError: ArgumentError: column name "Maximum line capacity (reactive power [KW])" not found in the data frame; existing most similar names are: "Maximum line capacity (active power [KW])"